<p style="font-size:36px;text-align:center"> <b>Self Drive Car Implemention Using Keras(Tensor FLow Framwork) Using "End-to-End Deep Learning for Self-Driving Cars" Network Architecture</b> </p>

<h1>1. Business Problem</h1>

<h2>1.1. Description</h2>

<p>In a new automotive application, we have used convolutional neural networks (CNNs) to map the raw pixels from a front-facing camera to the steering commands for a self-driving car. This powerful end-to-end approach means that with minimum training data from humans, the system learns to steer, with or without lane markings, on both local roads and highways. The system can also operate in areas with unclear visual guidance such as parking lots or unpaved roads.</p>

<p>In this Project i am going to implement Nvidia End-to-End Deep Learning for Self-Driving Cars" Network Architecture(https://arxiv.org/pdf/1604.07316.pdf) and around 45,000 Data-set(https://github.com/SullyChen/driving-datasets)</p>


<h2>1.2. Source/Useful Links</h2>

 Some articles and reference blogs about the problem statement
 <p>
1.appliedaicourse.com study videos

2.https://arxiv.org/pdf/1604.07316.pdf

3.https://devblogs.nvidia.com/deep-learning-self-driving-cars/

4.https://github.com/SullyChen/Autopilot-TensorFlow
<p>
  
<h2>1.3. Real-world/Business objectives and constraints.</h2>

* No low-latency requirement.
* Errors can be very costly.
*Regression problem

<h1>2. Deep Learning Problem Formulation</h1>

<h2>2.1. Data</h2>

<h3>2.1.1. Data Overview</h3>

- Source: https://drive.google.com/file/d/1PZWa6H0i1PCH9zuYcIh5Ouk_p-9Gh58B/view

<p>- Taking Same data (https://github.com/SullyChen/driving-datasets) SullyChen used in his/her project</p>
<p> Size: 25 minutes = 25*60*30 = 45,000 images ~ 2.3 GB</p>
<p>We have One data.txt file which contains image id and steering angle and all images

<p>Sample data.txt</p>
<p>
 0.jpg 0.000000
</p><p>
1.jpg 0.000000
</p><p>
2.jpg 0.000000
 </p>
 
<h2>2.2. Mapping the real-world problem to an DL problem</h2>
<h3>2.2.1. Type of Deep Learning Problem</h3>
As using giving images we have to predict steering angle (real number)=> Regression problem

<h3>2.2.2. Performance Metric</h3>

Metric(s):
<p>
MSE(Mean Squared Error):
These error functions average all the mistakes made for each prediction, and if the prediction is far from the true value, then this distance is made more evident by the squaring operation
</p>
<h2>
2.3. Train, CV and Test Datasets
</h2>
Split the dataset Temporal(Time based split) into three parts train, cross validation and test with 64%,16%, 20% of data respectively



<h1>Steps to Mount Google Drive with Google Colab </h1>
Going to Use Google Colab so first need to put data-set into one folder in google Drive.
Need to mount google drive with colab.
below stpes need to uncomment for first time 


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#unzip Autopilot folder as data is inside there
!unzip /content/drive/Car_data_set/Autopilot-TensorFlow-master.zip >temp.txt


In [0]:
!ls /content/Autopilot-TensorFlow-master/driving_dataset/


<h1>3. Exploratory Data Analysis</h1>
<h3>3.1. Import All required Classed first</h3>


In [5]:
import os, shutil
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from scipy import pi
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [23]:
#!rm -rf /content/Autopilot-TensorFlow-master/val_driving_dataset
#!rm -rf /content/Autopilot-TensorFlow-master/train_driving_dataset

<h1>4 Data preprocessing</h1>


As you know by now, data should be formatted into appropriately preprocessed floatingpoint
tensors before being fed into the network. Currently, the data sits on a drive as
JPEG files, so the steps for getting it into the network are roughly as follows:

1 Read the picture files.

2 Decode the JPG content to RGB grids of pixels.

3 Convert these into floating-point tensors.

4 Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know,
neural networks prefer to deal with small input values).

<p>Notekeras has build in class ImageDataGenerator, under keras.preprocessing.image to handle data preprocessing, will use that</p>

<h2>3.1. Reading Data</h2>
<p>reading data.txt file and spliting into Train and Test Data set(Training 80% & Test  20%)</p>
<p>Note:- 80% Training Data will split later 20% Validation data during keras fit operation</p>

In [9]:
xs = []
ys = []
original_data_path = r"/content/Autopilot-TensorFlow-master/driving_dataset"
train_data_path = r"/content/Autopilot-TensorFlow-master/train_driving_dataset"
val_data_path = r"/content/Autopilot-TensorFlow-master/val_driving_dataset"
#test_data_path = r"/content/Autopilot-TensorFlow-master/test_driving_dataset"

if not os.path.isdir(train_data_path):
    os.mkdir(train_data_path)
else:
  os.rmdir(train_data_path)
  os.mkdir(train_data_path)

if not os.path.isdir(val_data_path):
    os.mkdir(val_data_path)
else:
  os.rmdir(val_data_path)
  os.mkdir(val_data_path)
#read data.txt
with open(original_data_path +"/data.txt") as f:
    for line in f:
        xs.append(original_data_path+"/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * pi / 180)

#get number of images
num_images = len(xs)
val_start_point = int(len(xs) * 0.6)
val_end_point = int(len(xs) * 0.8)
                      
train_xs = xs[:val_start_point]
train_ys = ys[:val_start_point]
val_xs = xs[val_start_point : val_end_point]
val_ys = ys[val_start_point : val_end_point]
print("Total Number of Data-set: "+str(num_images))
print("Total Number of Train Data-set: "+str(len(train_xs)))
print("Total Number of Validation Data-set: "+str(len(val_xs)))


#Copy Train Images into train_data_path
for fname,str_angle in zip(train_xs,train_ys):
      dest_dir=os.path.join(train_data_path, str(str_angle))
      if not os.path.isdir(dest_dir):
          os.mkdir(dest_dir)             
      src = os.path.join(original_data_path, fname)
      shutil.copy(src, dest_dir)
      

#Copy Train Images into train_data_path
#for fname in val_xs:
#      src = os.path.join(original_data_path, fname)
#      shutil.copy(src, val_data_path)

for fname,str_angle in zip(val_xs,val_ys):
      dest_dir=os.path.join(val_data_path, str(str_angle))
      if not os.path.isdir(dest_dir):
          os.mkdir(dest_dir)             
      src = os.path.join(original_data_path, fname)
      shutil.copy(src, dest_dir)

#sanity check wheter train data is correct or not can compare with len(train_xs
print('total training images:', len(os.listdir(train_data_path)))
print('total Validation images:', len(os.listdir(val_data_path)))


Total Number of Data-set: 45406
Total Number of Train Data-set: 27243
Total Number of Validation Data-set: 9081
total training images: 1818
total training images: 1435


In [36]:
# Rescal all images by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_data_path,target_size=(66,200),batch_size=100,class_mode="sparse")

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = train_datagen.flow_from_directory(val_data_path,target_size=(66,200),batch_size=100,class_mode="sparse")

Found 27243 images belonging to 1818 classes.
Found 9081 images belonging to 1435 classes.


<h1>4.1. Build Model</h1>

<h3>4.1. Build Small Model</h3>
<p>Nvidia Network Model is very deep model to execute it will take more time(depends upon cpu and gpu configuration)</p>
</p>so quickly builing one small ConvNet model to get more confidence </p>

In [0]:
# define variables will use in model section

input_size = (66,200,3)

In [0]:
#create small model 
def small_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu',input_shape=input_size))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1))
  return model

In [28]:
model = small_model()

# check model summarry
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 198, 32)       896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 99, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 48, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 23, 128)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 21, 128)        147584    
__________

<h3>4.2 Configuring the model for training</h3>

In [0]:
model.compile(loss='mse',optimizer='adam',metrics=['acc'])

<h3>4.3 Fitting the model using a batch generator</h3>

In [0]:
history = model.fit_generator(train_generator,steps_per_epoch=300,epochs=300,validation_data=val_generator,validation_steps=100)

Epoch 1/300
300/300 [==============================] - 121s 403ms/step - loss: 25430.4647 - acc: 0.0039 - val_loss: 168285.2035 - val_acc: 0.0018
Epoch 2/300
293/300 [============================>.] - ETA: 2s - loss: 21946.0737 - acc: 0.0039

300/300 [==============================] - 118s 393ms/step - loss: 21894.9249 - acc: 0.0039 - val_loss: 150149.2886 - val_acc: 7.0133e-04
Epoch 3/300
300/300 [==============================] - 118s 393ms/step - loss: 19524.0506 - acc: 0.0048 - val_loss: 157891.6110 - val_acc: 0.0015
Epoch 4/300
 75/300 [======>.......................] - ETA: 59s - loss: 18594.6146 - acc: 0.0041

300/300 [==============================] - 118s 392ms/step - loss: 17628.4823 - acc: 0.0040 - val_loss: 177891.3949 - val_acc: 9.0171e-04
Epoch 5/300
300/300 [==============================] - 118s 392ms/step - loss: 15121.9826 - acc: 0.0044 - val_loss: 178385.8446 - val_acc: 0.0012
Epoch 6/300
 27/300 [=>............................] - ETA: 54s - loss: 15452.7774 - acc: 0.0063

300/300 [==============================] - 118s 394ms/step - loss: 14787.4510 - acc: 0.0050 - val_loss: 194993.2920 - val_acc: 0.0017
Epoch 7/300
300/300 [==============================] - 118s 393ms/step - loss: 13936.7165 - acc: 0.0044 - val_loss: 172478.9625 - val_acc: 0.0010
Epoch 8/300
 20/300 [=>............................] - ETA: 46s - loss: 15647.5982 - acc: 0.0030

300/300 [==============================] - 118s 392ms/step - loss: 12494.3891 - acc: 0.0056 - val_loss: 162134.5476 - val_acc: 8.0152e-04
Epoch 9/300
300/300 [==============================] - 118s 393ms/step - loss: 12236.6064 - acc: 0.0050 - val_loss: 169098.2022 - val_acc: 0.0021
Epoch 10/300
 15/300 [>.............................] - ETA: 34s - loss: 10728.5211 - acc: 0.0047

...

300/300 [==============================] - 119s 396ms/step - loss: 8031.5575 - acc: 0.0057 - val_loss: 172916.5470 - val_acc: 9.0171e-04
Epoch 17/300
300/300 [==============================] - 118s 394ms/step - loss: 8156.7418 - acc: 0.0068 - val_loss: 162766.6872 - val_acc: 0.0011
Epoch 18/300
 17/300 [>.............................] - ETA: 40s - loss: 7524.9554 - acc: 0.0047

300/300 [==============================] - 119s 396ms/step - loss: 7378.0248 - acc: 0.0066 - val_loss: 177290.1594 - val_acc: 9.0171e-04
Epoch 19/300
300/300 [==============================] - 120s 399ms/step - loss: 7565.9947 - acc: 0.0059 - val_loss: 189597.7276 - val_acc: 7.0133e-04
Epoch 20/300
 15/300 [>.............................] - ETA: 35s - loss: 7766.9461 - acc: 0.0053

300/300 [==============================] - 118s 392ms/step - loss: 6807.6867 - acc: 0.0064 - val_loss: 160708.8089 - val_acc: 8.0152e-04
Epoch 21/300
300/300 [==============================] - 119s 397ms/step - loss: 6869.2537 - acc: 0.0074 - val_loss: 168049.9549 - val_acc: 0.0014
Epoch 22/300
 16/300 [>.............................] - ETA: 37s - loss: 6377.8509 - acc: 0.0075

300/300 [==============================] - 119s 395ms/step - loss: 6481.2577 - acc: 0.0067 - val_loss: 190700.3945 - val_acc: 0.0011
Epoch 23/300
300/300 [==============================] - 118s 393ms/step - loss: 6008.8979 - acc: 0.0072 - val_loss: 173052.1078 - val_acc: 0.0013
Epoch 24/300
 20/300 [=>............................] - ETA: 46s - loss: 5847.0306 - acc: 0.0075

130/300 [============>.................] - ETA: 47s - loss: 5981.0356 - acc: 0.0075

<h3>4.4 Save Model</h3>

In [0]:
model.save("self_car_small_model.h5")

<h3>4.5 Displaying curves of loss and accuracy during training</h3>

In [0]:
def display_perfomance():
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(acc) + 1)
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()

<h3>5.1. Build Nvidia Style Model</h3>
<p>Network Architecture of Nvidia, i wlll follow and will do some experiment on top up that link will add Dropout,BatchNormalization as next task etc</p>

In [0]:
# define variables will use in model section

input_size = (66,200,3)

In [0]:
#create small model 
def create_model():
  model = Sequential()
  model.add(conv2D(24, (5, 5), activation='relu',input_shape=input_size)
  model.add(Conv2D(36, (5, 5), activation='relu'))
  model.add(Conv2D(48, (5, 5), activation='relu'))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1)
  return model

12813


In [0]:
model = create_model()

# check model summarry
model.summary()

<h3>5.2 Configuring the model for training</h3>

In [0]:
model.compile(loss='mse',optimizer='adam',metrics=['acc'])

<h3>5.3 Fitting the model using a batch generator</h3>

In [0]:
history = model.fit_generator(train_generator,steps_per_epoch=100,epochs=30,validation_data=validation_generator,validation_steps=50)

<h3>5.4 Save Model</h3>

In [0]:
model.save("nvidia_self_car_model.h5")